In [1]:
from load_data_and_augmentations import *

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
plt.rcParams['figure.figsize'] = (20,10)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 20}
plt.rc('font', **font)

In [11]:
tensor_transform = get_tensor_transform('Kinetics', True)
train_spat_transform = get_spatial_transform(2)
train_temp_transform = get_temporal_transform()
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

root_dir = '/media/scratch/astamoulakatos/centre_Ch2/'
df = pd.read_csv('../important_csvs/more_balanced_dataset/more_balanced_stratified.csv')


In [12]:
df = df.sample(frac=0.01)

In [13]:
df = df.reset_index(drop=True)

In [14]:
df

,event_path,number_of_frames,exposure,burial,field_joint,anode,free_span,label,fold,stratify_group
0,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,308,1,0,0,0,0,exp,4,exp_1
1,/media/scratch/astamoulakatos/centre_Ch2/exp_f...,106,1,0,0,0,1,exp_fs,0,exp_fs_0
2,/media/scratch/astamoulakatos/centre_Ch2/exp_f...,121,1,0,1,0,0,exp_fj,4,exp_fj_1
3,/media/scratch/astamoulakatos/centre_Ch2/exp_f...,331,1,0,0,0,1,exp_fs,1,exp_fs_0
4,/media/scratch/astamoulakatos/centre_Ch2/bur/S...,400,0,1,0,0,0,bur,2,bur_1
5,/media/scratch/astamoulakatos/centre_Ch2/exp_f...,92,1,0,1,0,0,exp_fj,0,exp_fj_0
6,/media/scratch/astamoulakatos/centre_Ch2/bur/S...,400,0,1,0,0,0,bur,4,bur_1
7,/media/scratch/astamoulakatos/centre_Ch2/exp_f...,97,1,0,1,0,0,exp_fj,2,exp_fj_0
8,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,0,exp_1
9,/media/scratch/astamoulakatos/centre_Ch2/exp_f...,138,1,0,1,0,0,exp_fj,0,exp_fj_1


In [15]:
df_train = get_df(df, 50, True, False, False)
class_image_paths, end_idx = get_indices(df_train, root_dir)
train_loader = get_loader(16, 64, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, False, False)
df_valid = get_df(df, 50, False, True, False)
class_image_paths, end_idx = get_indices(df_valid, root_dir)
valid_loader = get_loader(16, 64, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, False)
df_test = get_df(df, 50, False, False, True)
class_image_paths, end_idx = get_indices(df_test, root_dir)
test_loader = get_loader(16, 64, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, False)


In [16]:
end_idx

tensor([  0, 106, 506, 598, 736])

In [19]:
seq_length = 16

In [20]:
indices = []
for i in range(len(end_idx) - 1):
    start = end_idx[i]
    end = end_idx[i + 1] - seq_length
    if start > end:
        indices.append(torch.arange(end, start))
    else:
        indices.append(torch.arange(start, end))
indices = torch.cat(indices)
#self.indices = indices

In [21]:
len(indices)

672

In [22]:
indices = indices[torch.randperm(len(indices))]
t = iter(indices.tolist())

In [23]:
indices

tensor([674, 286, 521,  44, 337, 253, 306,  65, 650, 537, 579, 439, 228, 660,
        291, 670, 127, 692, 193,  41, 190, 468, 472, 277, 248, 300, 526, 258,
        121,  10, 168, 157, 462,  75, 227, 130, 192, 371, 265, 419, 335, 331,
        197, 536, 613, 186,  17,  42,   5, 391,  57, 523, 272, 632, 413, 199,
        322, 421, 123, 214, 708, 298, 424, 343, 397, 614, 133, 563, 691, 169,
        219, 285, 639, 651, 489,  69,  16, 658, 191, 259,  66, 690,  27, 360,
         47, 178, 624, 545, 274, 115, 129, 138,  81, 681, 223, 460, 454, 520,
        486, 365, 188, 534, 464, 631,   6, 225, 513, 652, 354, 342, 210, 389,
        351, 525, 334, 363, 473, 324, 126, 398, 701, 260, 403, 176, 679, 605,
        428, 352, 560, 411, 109, 339,  87, 255, 108, 418,  83, 321, 326, 142,
        474, 390, 557, 250, 656, 532, 438, 330, 538, 689, 252, 160,  39, 350,
        364, 139,  61,  67, 141, 488, 240, 231, 128,  55, 278,  19, 446, 185,
          1, 393, 167, 669, 198,  49, 512, 470, 707, 122, 239,  

In [17]:
sampler = MySampler(end_idx, 16)

In [18]:
sampler.indices

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89, 106, 107, 108, 109, 110, 111, 112, 113,
        114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
        128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
        142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
        184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 1